In [1]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

import platform as plat
import os
import time

import keras as kr
import numpy as np
import random
import pdb

import tensorflow as tf
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Input, Reshape, BatchNormalization # , Flatten
from keras.layers import Lambda, TimeDistributed, Activation,Conv2D, MaxPooling2D,GRU, Bidirectional #, Merge
from keras.layers.merge import add, concatenate
from keras import backend as K
from keras.optimizers import SGD, Adadelta, Adam

# 指定第一块GPU可用
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
config = tf.ConfigProto()
config.gpu_options.allow_growth=True   #不全部占满显存, 按需分配
# config.gpu_options.per_process_gpu_memory_fraction = 0.3
sess = tf.Session(config=config)
K.set_session(sess)

Using TensorFlow backend.


In [2]:
from utils.common import *
from utils.wav_preprocess import compute_freq_feature
from speech_data import test_data_generator

py2id_dict = pinyin2id()
id2py_dict = dict(zip(py2id_dict.values(), py2id_dict.keys()))

## 搭建模型

In [3]:
MODEL_LOGS_DIR_NAME = "logs_speechRec_1"

class SpeechRecognitionModelV1():
    '''
    定义CNN/LSTM/CTC模型，使用函数式模型
    输入层：200维的特征值序列，一条语音数据的最大长度设为1600（大约16s）
    隐藏层：卷积池化层，卷积核大小为3x3，池化窗口大小为2
    隐藏层：全连接层
    输出层：全连接层，神经元数量为self.MS_OUTPUT_SIZE，使用softmax作为激活函数，
    CTC层：使用CTC的loss作为损失函数，实现连接性时序多输出

    '''

    def __init__(self):
        '''
        初始化
        默认输出的拼音的表示大小是1423，即1423个拼音+1个空白块
        '''
        self.MS_OUTPUT_SIZE = 1423 + 1  # 神经网络最终输出的每一个字符向量维度的大小
        # self.BATCH_SIZE = BATCH_SIZE # 一次训练的batch
        self.label_max_string_length = 64
        self.AUDIO_LENGTH = 16000
        self.AUDIO_FEATURE_LENGTH = 200

        self.model, self.ctc_model = self._model_init()

    def _model_init(self):
        input_data = Input(name="the_inputs", shape=(None, self.AUDIO_FEATURE_LENGTH, 1))

        x = Conv2D(32, (3, 3), use_bias=False, activation='relu', padding='same', kernel_initializer='he_normal')(
            input_data)
        x = BatchNormalization()(x)
        x = Conv2D(32, (3, 3), use_bias=True, activation='relu', padding='same', kernel_initializer='he_normal')(x)
        x = BatchNormalization()(x)
        x = MaxPooling2D(pool_size=2, strides=None, padding='valid')(x)

        x = Conv2D(64, (3, 3), use_bias=True, activation='relu', padding='same', kernel_initializer='he_normal')(x)
        x = BatchNormalization()(x)
        x = Conv2D(64, (3, 3), use_bias=True, activation='relu', padding='same', kernel_initializer='he_normal')(x)
        x = BatchNormalization()(x)

        x = Conv2D(64, (3, 3), use_bias=True, activation='relu', padding='same', kernel_initializer='he_normal')(x)
        x = BatchNormalization()(x)
        x = Conv2D(64, (3, 3), use_bias=True, activation='relu', padding='same', kernel_initializer='he_normal')(x)
        x = BatchNormalization()(x)
        x = MaxPooling2D(pool_size=2, strides=None, padding='valid')(x)

        x = Conv2D(128, (3, 3), use_bias=True, activation='relu', padding='same', kernel_initializer='he_normal')(x)
        x = BatchNormalization()(x)
        x = Conv2D(128, (3, 3), use_bias=True, activation='relu', padding='same', kernel_initializer='he_normal')(x)
        x = BatchNormalization()(x)

        x = Conv2D(128, (3, 3), use_bias=True, activation='relu', padding='same', kernel_initializer='he_normal')(x)
        x = BatchNormalization()(x)
        x = Conv2D(128, (3, 3), use_bias=True, activation='relu', padding='same', kernel_initializer='he_normal')(x)
        x = BatchNormalization()(x)

        x = Conv2D(128, (3, 3), use_bias=True, activation='relu', padding='same', kernel_initializer='he_normal')(x)
        x = BatchNormalization()(x)
        x = Conv2D(128, (3, 3), use_bias=True, activation='relu', padding='same', kernel_initializer='he_normal')(x)
        x = BatchNormalization()(x)
        x = MaxPooling2D(pool_size=2, strides=None, padding='valid')(x)

        # 200 / 8 * 128 = 3200
        # 因为DFT抽样点数必须为2的整数次幂，经过3层maxpooling层，要求音频数据的每个维度需要能够被8整除
        x = Reshape(target_shape=(-1, 3200))(x)

        # x = Dropout(0.2)(x)
        x = Dense(128, activation='relu', use_bias=True, kernel_initializer='he_normal')(x)
        x = BatchNormalization()(x)

        gru_units = 128
        # 创建一个双向GRU，看看是否能增加精度？
        #         gru_1a = GRU(gru_units, return_sequences=True, kernel_initializer='he_normal', name='gru_1a')(x)
        #         gru_1b = GRU(gru_units, return_sequences=True, go_backwards=True, kernel_initializer='he_normal', name='gru_1b')(x)
        x = Bidirectional(GRU(gru_units, return_sequences=True, kernel_initializer='he_normal', name='gru_1'))(x)
        x = BatchNormalization()(x)

        x = Bidirectional(GRU(gru_units, return_sequences=True, kernel_initializer='he_normal', name='gru_2'))(x)
        x = BatchNormalization()(x)

        x = Dense(128, activation='relu', use_bias=True, kernel_initializer='he_normal')(x)
        x = BatchNormalization()(x)

        x = Dense(self.MS_OUTPUT_SIZE, use_bias=True, kernel_initializer='he_normal')(x)

        y_pred = Activation('softmax', name='y_pred_activation')(x)
        model_data = Model(inputs=input_data, outputs=y_pred)

        labels = Input(name='the_labels', shape=[None], dtype='float32')
        input_length = Input(name='input_length', shape=[1], dtype='int64')
        label_length = Input(name='label_length', shape=[1], dtype='int64')
        loss_out = Lambda(self._ctc_batch_cost_func, output_shape=(1,), name='ctc_loss')(
            [y_pred, labels, input_length, label_length])

        ctc_model = Model(inputs=[input_data, labels, input_length, label_length], outputs=loss_out)
        ctc_model.summary()

        optimizer = Adam(learning_rate=0.0003, beta_1=0.9, beta_2=0.999, decay=0.0, epsilon=10e-8)
        ctc_model.compile(optimizer=optimizer, loss={'ctc_loss': lambda y_true, y_pred: y_pred}, metrics=['acc'])

        # captures output of softmax so we can decode the output during visualization
        #         test_func = K.function([input_data], [self.y_pred])
        #         pdb.set_trace()

        # print('[*提示] 创建模型成功，模型编译成功')
        print('[*Info] Create Model Successful, Compiles Model Successful. ')
        return model_data, ctc_model

    def _ctc_batch_cost_func(self, args):
        y_pred, labels, input_length, label_length = args

        #         pdb.set_trace()
        y_pred = y_pred[:, :, :]
        return K.ctc_batch_cost(labels, y_pred, input_length, label_length)

    def predict(self, data_input, input_len):
        '''
        预测结果，返回拼音对应的id列表
        :param data_input: 输入的音频数据
        :param input_len: 音频的长度
        :return:
        '''
        batch_size = 1

        base_pred = self.model.predict(data_input, batch_size)
#         print("base_pred-pre:", base_pred)
        base_pred = base_pred[:, :, :]
#         print("base_pred:", base_pred)

        r = K.ctc_decode(base_pred, input_len, greedy=True, beam_width=100, top_paths=1)
#         print("r:", r)
        r1 = K.get_value(r[0][0])
#         print("r1:", r1)

        return r1

    def recognize(self, wav_file):
        '''
        wav 文件识别出拼音
        :param wav_file:
        :return:
        '''
        data_input = compute_freq_feature(wav_file)
        input_length = len(data_input) // 8

        data_input = np.array(data_input, dtype=np.float)
        data_input = np.reshape(data_input.shape[0], data_input.shape[1], 1)

        r1 = self.predict(data_input, input_length)
        recognize_pingyin_list = [id2py_dict[id] for id in r1]
        return recognize_pingyin_list

    def test_model(self, wav_label_list, data_batch_size=4):
        '''
        验证模型的水准
        :param wav_data_path:  验证集wav数据集合
        :param labels_data_path: 对应的拼音labels id集合
        :param data_batch_size: 验证集的数量
        :return:
        '''

        for next_index in range(data_batch_size):
            batch = data_generator(wav_label_list, 1)

            word_error_num = 0
            word_total_num = 0
            input, output = next(batch)

            y_predict = self.predict(input, input['input_length'])
            label = input['the_labels']

            real_str = ''.join([id2py_dict[id] for id in label[0].tolist()])
            predict_str = ''.join([id2py_dict[id] for id in y_predict[0].tolist()])

            words_num = label.shape[1]
            word_total_num += words_num
            distance = calculate_sequence_edit_distance(label[0].tolist(), y_predict[0].tolist())
            if distance <= words_num:
                word_error_num += distance  # 使用编辑距离作为错误字数
            else:  # 否则肯定是增加了一堆乱七八糟的奇奇怪怪的字,就直接加句子本来的总字数就好了
                word_error_num += words_num

            print("[原本语音内容]：", real_str)
            print('[**预测结果**]：', predict_str)
            print("============********============" + "\n")

        acc = (word_total_num - word_error_num) / word_total_num
        print('*本轮语音测试准确率：', str(acc))

    def load_last_weights(self):
        sorted_model_list = sorted(glob.glob(os.path.join(MODEL_LOGS_DIR_NAME, '*.model')),
                                   key=lambda x: time.localtime(os.path.getmtime(x)),
                                   reverse=True)
        sorted_ctc_model_list = sorted(glob.glob(os.path.join(MODEL_LOGS_DIR_NAME, '*.model.base')),
                                       key=lambda x: time.localtime(os.path.getmtime(x)),
                                       reverse=True)

        self.model.load_weights(sorted_model_list[0])
        self.ctc_model.load_weights(sorted_ctc_model_list[0])

## 训练模型

In [4]:
from speech_data import *
from utils.wav_preprocess import *
# from speechRecognitionModelV1 import SpeechRecognitionModelV1

m = SpeechRecognitionModelV1()

epochs = 100
batch_size = 4

# train_wav_list = get_thchs30_data(wav_base_path="H:\\PycharmProjects\\dataset\\data_thchs30\\train")
# validation_wav_list = get_thchs30_data(wav_base_path="H:\\PycharmProjects\\dataset\\data_thchs30\\dev")

train_wav_list = load_various_wav_train_data()
validation_wav_list = load_various_wav_dev_data()
batch_num = len(train_wav_list) // batch_size
# val_batch_num = int(batch_num * 0.2)

train_batch = data_generator(train_wav_list, batch_size)
# validation_batch = data_generator(validation_wav_list, batch_size)

m.load_last_weights()

for i in range(epochs):
    print("Begin epoch:", i+1)
    train_batch = data_generator(train_wav_list, batch_size)
    history = m.ctc_model.fit_generator(train_batch,
                                        steps_per_epoch=batch_num,
                                        epochs=1)
    m.test_model(wav_label_list=validation_wav_list, data_batch_size=32)

    m.model.save_weights(os.path.join(MODEL_LOGS_DIR_NAME, str(i) +'_steps_SpeechRecognitionModelV1.model'))
    m.ctc_model.save_weights(os.path.join(MODEL_LOGS_DIR_NAME, str(i) +'_steps_SpeechRecognitionModelV1.model.base'))



Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
the_inputs (InputLayer)         (None, None, 200, 1) 0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, None, 200, 32 288         the_inputs[0][0]                 
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, None, 200, 32 128         conv2d_1[0][0]                   
__________________________________________________________________________________________________
conv2d_2 (Conv2D)               (None, None, 200, 32 9248        batch_normalization_1[0][0]    

dense_3 (Dense)                 (None, None, 1424)   183696      batch_normalization_16[0][0]     
__________________________________________________________________________________________________
y_pred_activation (Activation)  (None, None, 1424)   0           dense_3[0][0]                    
__________________________________________________________________________________________________
the_labels (InputLayer)         (None, None)         0                                            
__________________________________________________________________________________________________
input_length (InputLayer)       (None, 1)            0                                            
__________________________________________________________________________________________________
label_length (InputLayer)       (None, 1)            0                                            
__________________________________________________________________________________________________
ctc_loss (

 204/2500 [=>............................] - ETA: 4:05:19 - loss: 0.0269 - acc: 1.00 - ETA: 2:30:29 - loss: 0.0260 - acc: 1.00 - ETA: 1:53:49 - loss: 0.0282 - acc: 1.00 - ETA: 1:35:54 - loss: 0.1202 - acc: 0.93 - ETA: 1:26:07 - loss: 0.1666 - acc: 0.90 - ETA: 1:19:07 - loss: 0.1432 - acc: 0.91 - ETA: 1:15:14 - loss: 0.7019 - acc: 0.89 - ETA: 1:11:12 - loss: 0.6269 - acc: 0.90 - ETA: 1:09:20 - loss: 0.5612 - acc: 0.91 - ETA: 1:07:47 - loss: 0.9751 - acc: 0.87 - ETA: 1:05:48 - loss: 1.2398 - acc: 0.86 - ETA: 1:01:46 - loss: 1.1399 - acc: 0.87 - ETA: 1:00:43 - loss: 1.3646 - acc: 0.86 - ETA: 59:51 - loss: 7.5009 - acc: 0.8571 - ETA: 59:25 - loss: 7.0032 - acc: 0.86 - ETA: 58:38 - loss: 6.5709 - acc: 0.87 - ETA: 57:52 - loss: 6.1874 - acc: 0.88 - ETA: 57:50 - loss: 5.8452 - acc: 0.88 - ETA: 57:45 - loss: 5.5509 - acc: 0.88 - ETA: 55:53 - loss: 5.2879 - acc: 0.87 - ETA: 54:02 - loss: 5.0379 - acc: 0.88 - ETA: 53:43 - loss: 4.8118 - acc: 0.88 - ETA: 52:04 - loss: 4.7803 - acc: 0.88 - ETA: 52

 408/2500 [===>..........................] - ETA: 26:39 - loss: 1.1324 - acc: 0.89 - ETA: 26:35 - loss: 1.1283 - acc: 0.89 - ETA: 26:31 - loss: 1.1260 - acc: 0.89 - ETA: 26:28 - loss: 1.1212 - acc: 0.89 - ETA: 26:24 - loss: 1.1160 - acc: 0.89 - ETA: 26:20 - loss: 1.1116 - acc: 0.89 - ETA: 26:17 - loss: 1.1084 - acc: 0.89 - ETA: 26:13 - loss: 1.1137 - acc: 0.89 - ETA: 26:10 - loss: 1.1086 - acc: 0.89 - ETA: 26:07 - loss: 1.1036 - acc: 0.89 - ETA: 26:03 - loss: 1.0986 - acc: 0.89 - ETA: 26:00 - loss: 1.0939 - acc: 0.89 - ETA: 25:57 - loss: 1.0921 - acc: 0.89 - ETA: 25:54 - loss: 1.0888 - acc: 0.89 - ETA: 25:50 - loss: 1.0847 - acc: 0.89 - ETA: 25:47 - loss: 1.0800 - acc: 0.89 - ETA: 25:50 - loss: 1.0782 - acc: 0.89 - ETA: 25:46 - loss: 1.0783 - acc: 0.89 - ETA: 25:42 - loss: 1.0736 - acc: 0.89 - ETA: 25:40 - loss: 1.0692 - acc: 0.89 - ETA: 25:36 - loss: 1.0648 - acc: 0.89 - ETA: 25:32 - loss: 1.0604 - acc: 0.89 - ETA: 25:28 - loss: 1.0585 - acc: 0.89 - ETA: 25:25 - loss: 1.0553 - acc: 0.

 612/2500 [======>.......................] - ETA: 19:46 - loss: 0.9406 - acc: 0.88 - ETA: 19:44 - loss: 0.9384 - acc: 0.88 - ETA: 19:43 - loss: 0.9366 - acc: 0.88 - ETA: 19:41 - loss: 0.9344 - acc: 0.88 - ETA: 19:40 - loss: 0.9323 - acc: 0.88 - ETA: 19:38 - loss: 0.9308 - acc: 0.88 - ETA: 19:37 - loss: 0.9387 - acc: 0.88 - ETA: 19:36 - loss: 0.9371 - acc: 0.88 - ETA: 19:35 - loss: 0.9352 - acc: 0.88 - ETA: 19:33 - loss: 0.9352 - acc: 0.88 - ETA: 19:32 - loss: 0.9333 - acc: 0.88 - ETA: 19:35 - loss: 0.9311 - acc: 0.88 - ETA: 19:34 - loss: 0.9482 - acc: 0.88 - ETA: 19:32 - loss: 0.9460 - acc: 0.88 - ETA: 19:31 - loss: 0.9439 - acc: 0.88 - ETA: 19:30 - loss: 0.9419 - acc: 0.88 - ETA: 19:29 - loss: 0.9408 - acc: 0.88 - ETA: 19:27 - loss: 0.9483 - acc: 0.88 - ETA: 19:26 - loss: 0.9463 - acc: 0.88 - ETA: 19:25 - loss: 0.9447 - acc: 0.88 - ETA: 19:23 - loss: 0.9428 - acc: 0.88 - ETA: 19:22 - loss: 0.9410 - acc: 0.88 - ETA: 19:21 - loss: 0.9395 - acc: 0.88 - ETA: 19:20 - loss: 0.9374 - acc: 0.

...

2453/2500 [============================>.] - ETA: 1:47 - loss: 0.9867 - acc: 0.856 - ETA: 1:46 - loss: 0.9864 - acc: 0.856 - ETA: 1:46 - loss: 0.9859 - acc: 0.856 - ETA: 1:46 - loss: 0.9857 - acc: 0.856 - ETA: 1:45 - loss: 0.9856 - acc: 0.856 - ETA: 1:45 - loss: 0.9854 - acc: 0.856 - ETA: 1:44 - loss: 0.9850 - acc: 0.856 - ETA: 1:44 - loss: 0.9846 - acc: 0.856 - ETA: 1:44 - loss: 0.9845 - acc: 0.856 - ETA: 1:43 - loss: 0.9859 - acc: 0.855 - ETA: 1:43 - loss: 0.9891 - acc: 0.855 - ETA: 1:42 - loss: 0.9892 - acc: 0.855 - ETA: 1:42 - loss: 0.9888 - acc: 0.855 - ETA: 1:41 - loss: 0.9885 - acc: 0.855 - ETA: 1:41 - loss: 0.9882 - acc: 0.855 - ETA: 1:41 - loss: 0.9878 - acc: 0.855 - ETA: 1:40 - loss: 0.9874 - acc: 0.855 - ETA: 1:40 - loss: 0.9870 - acc: 0.855 - ETA: 1:39 - loss: 0.9867 - acc: 0.855 - ETA: 1:39 - loss: 0.9879 - acc: 0.855 - ETA: 1:38 - loss: 0.9875 - acc: 0.855 - ETA: 1:38 - loss: 0.9873 - acc: 0.855 - ETA: 1:38 - loss: 0.9869 - acc: 0.855 - ETA: 1:37 - loss: 0.9865 - acc: 0.8

2500/2500 [==============================] - ETA: 19s - loss: 0.9811 - acc: 0.85 - ETA: 18s - loss: 0.9824 - acc: 0.85 - ETA: 18s - loss: 0.9837 - acc: 0.85 - ETA: 18s - loss: 0.9833 - acc: 0.85 - ETA: 17s - loss: 0.9829 - acc: 0.85 - ETA: 17s - loss: 0.9825 - acc: 0.85 - ETA: 16s - loss: 0.9822 - acc: 0.85 - ETA: 16s - loss: 0.9819 - acc: 0.85 - ETA: 16s - loss: 0.9817 - acc: 0.85 - ETA: 15s - loss: 0.9814 - acc: 0.85 - ETA: 15s - loss: 0.9810 - acc: 0.85 - ETA: 14s - loss: 0.9806 - acc: 0.85 - ETA: 14s - loss: 0.9803 - acc: 0.85 - ETA: 13s - loss: 0.9816 - acc: 0.85 - ETA: 13s - loss: 0.9812 - acc: 0.85 - ETA: 13s - loss: 0.9808 - acc: 0.85 - ETA: 12s - loss: 0.9805 - acc: 0.85 - ETA: 12s - loss: 0.9803 - acc: 0.85 - ETA: 11s - loss: 0.9802 - acc: 0.85 - ETA: 11s - loss: 0.9798 - acc: 0.85 - ETA: 10s - loss: 0.9794 - acc: 0.85 - ETA: 10s - loss: 0.9795 - acc: 0.85 - ETA: 10s - loss: 0.9807 - acc: 0.85 - ETA: 9s - loss: 0.9803 - acc: 0.8565 - ETA: 9s - loss: 0.9800 - acc: 0.856 - ETA:

[原本语音内容]： jing3cha2ti3yu4wu3xiang4bi3sai4fen1bie2wei2tiao4yuan3qian1qiu2you2yong3she4ji1he2yue4ye3pao3
[**预测结果**]： jing3cha2ti3yu4wu3xiang4bi3sai4fen1bie2wei2tiao4yuan3qian1qiu2you2yong3she4ji1he2yue4ye3pao3
============********============

[原本语音内容]： jing1guo5shang4shang4xia4xia4de5gong4tong2nu3li4e4zhi4le5wai4zi1e2xia4jiang4de5qu1shi4chu1xian4wai4zi1e2da4fu2du4hui2sheng1de5ju2mian4
[**预测结果**]： jing1guo5shang4shang4xia4xia4de5gong4tong2nu3li4e4zhi4le5wai4zi1e2xia4jiang4de5qu1shi4chu1xian4wai4zi1e2da4fu2du4hui2sheng1de5ju2mian4
============********============

[原本语音内容]： zhe4yang4de5xiu1yang3de5yan3yuan2ji2shi3zheng4zou3hong2ye3bu2guo4ru2guo4yan3yun2yan1er2yi3cheng2bu5liao3zhen1zheng4yi4yi4shang4de5yi4shu4jia1
[**预测结果**]： zhe4yang4de5xiu1yang3de5yan3yuan2ji2shi3zheng4zou3hong2ye3bu2mo4ru2guo4yan3yun2yan1er2yi3cheng2bu5liao3zhen1zheng4yi4shang4de5yi4shu4jia1
============********============

[原本语音内容]： zhe4ci4quan2guo2qing1nian2pai2qiu2lian2sai4gong4she4tian1jin1zhou1shan1wu3han4san1ge5s

 105/2500 [>.............................] - ETA: 27:45 - loss: 0.0277 - acc: 1.00 - ETA: 21:58 - loss: 0.0651 - acc: 1.00 - ETA: 19:52 - loss: 0.0548 - acc: 1.00 - ETA: 19:53 - loss: 0.0476 - acc: 1.00 - ETA: 18:52 - loss: 0.0601 - acc: 1.00 - ETA: 18:27 - loss: 0.2275 - acc: 0.95 - ETA: 18:04 - loss: 0.2035 - acc: 0.96 - ETA: 17:35 - loss: 0.1851 - acc: 0.96 - ETA: 18:05 - loss: 0.1680 - acc: 0.97 - ETA: 18:03 - loss: 0.1616 - acc: 0.97 - ETA: 17:47 - loss: 0.1495 - acc: 0.97 - ETA: 18:04 - loss: 0.1689 - acc: 0.95 - ETA: 18:31 - loss: 0.4710 - acc: 0.94 - ETA: 18:31 - loss: 0.4400 - acc: 0.94 - ETA: 18:33 - loss: 0.4180 - acc: 0.95 - ETA: 18:20 - loss: 0.4005 - acc: 0.95 - ETA: 18:19 - loss: 0.3930 - acc: 0.94 - ETA: 18:12 - loss: 0.3992 - acc: 0.93 - ETA: 18:09 - loss: 0.3801 - acc: 0.93 - ETA: 18:05 - loss: 0.3828 - acc: 0.91 - ETA: 18:06 - loss: 0.3660 - acc: 0.91 - ETA: 18:04 - loss: 0.3505 - acc: 0.92 - ETA: 17:58 - loss: 0.3385 - acc: 0.92 - ETA: 17:50 - loss: 0.3253 - acc: 0.

ResourceExhaustedError: 2 root error(s) found.
  (0) Resource exhausted: OOM when allocating tensor with shape[134,4,1424] and type float on /job:localhost/replica:0/task:0/device:CPU:0 by allocator cpu
	 [[{{node ctc_loss/CTCLoss}}]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.

	 [[training/Adam/gradients/ctc_loss/CTCLoss_grad/mul/_1309]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.

  (1) Resource exhausted: OOM when allocating tensor with shape[134,4,1424] and type float on /job:localhost/replica:0/task:0/device:CPU:0 by allocator cpu
	 [[{{node ctc_loss/CTCLoss}}]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.

0 successful operations.
0 derived errors ignored.